In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("datasets/covid_philippines.csv")

In [5]:
df.head()

,Unnamed: 0,video_id,video_title,description,video_dop,view_count,like_count,comment_count,channel_id,channel_title,channel_dop,sub_count,total_videos
0,0,m3P-bmt3Uqw,JN.1 COVID-19 subvariant causing spike in cases,'An infectious disease expert says the new COV...,2023-12-25,6494,45,18.0,UCj6spMO3ybZPobE0T5perHA,CNN Philippines,2015-03-16,1390000,22740
1,1,sYI97jv-pZg,"PH records 2,725 new COVID cases from Dec. 12 ...",'The World Tonight: The daily average of the P...,2023-12-18,1982,14,3.0,UCvi6hEzLM-Z_unKPSuuzKvg,ANC 24/7,2010-01-29,1500000,70070
2,2,3YFpjgIQqEo,WATCH: DOH Usec. Tayag on rise of COVID-19 cas...,'Dateline Philippines: Karmina Constantino tal...,2023-12-07,9353,80,17.0,UCvi6hEzLM-Z_unKPSuuzKvg,ANC 24/7,2010-01-29,1500000,70070
3,3,DWxIvQlpJK8,Metro Manila to be placed on lockdown due to c...,'Subscribe: https://www.youtube.com/@Rappler/\...,2020-03-12,104353,736,18.0,UCdnZdQxYXnbN4uWJg96oGxw,Rappler,2011-12-02,1930000,50275
4,4,aLZ85hb4wjE,Normally crowded streets of Philippine capital...,'Subscribe to our YouTube channel for free her...,2020-04-02,323096,3283,619.0,UC4SUWizzKc1tptprBkWjX2Q,South China Morning Post,2007-01-18,3790000,16907


In [7]:
df[['like_count', 'comment_count']].describe()

,like_count,comment_count
count,400.000000,398.000000
mean,473.962500,183.587940
std,1164.573008,522.077111
min,2.000000,0.000000
25%,54.500000,13.250000
50%,146.500000,51.000000
75%,387.500000,166.000000
max,12843.000000,6508.000000
